# AI-Ethica Quick Start Guide

This notebook demonstrates the core features of AI-Ethica: bias detection, fairness metrics, and transparency analysis.


In [ ]:
# Install ai-ethica (uncomment if needed)
# !pip install ai-ethica


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Import AI-Ethica modules
from ai_ethica import BiasDetector, FairnessMetrics, TransparencyAnalyzer


## 1. Bias Detection

Detect bias in datasets across protected attributes.


In [ ]:
# Create sample dataset with potential bias
np.random.seed(42)
n_samples = 1000
data = pd.DataFrame({
    'feature1': np.random.randn(n_samples),
    'feature2': np.random.randn(n_samples),
    'gender': np.random.choice(['M', 'F'], n_samples, p=[0.7, 0.3]),  # Imbalanced
    'race': np.random.choice(['A', 'B', 'C'], n_samples, p=[0.5, 0.3, 0.2]),
    'target': np.random.choice([0, 1], n_samples, p=[0.6, 0.4])
})

# Initialize bias detector
detector = BiasDetector()

# Analyze bias
bias_report = detector.analyze(
    data=data,
    protected_attributes=['gender', 'race'],
    target_column='target'
)

print(f"Dataset size: {bias_report['dataset_size']}")
print(f"\nBias Metrics:")
for attr, metrics in bias_report['bias_metrics'].items():
    print(f"\n  {attr}:")
    rep_bias = metrics['representation_bias']
    print(f"    Representation disparity ratio: {rep_bias['disparity_ratio']:.2f}")
    print(f"    Is balanced: {rep_bias['is_balanced']}")

print(f"\nRecommendations:")
for rec in bias_report['recommendations']:
    print(f"  - {rec}")


## 2. Fairness Metrics

Calculate fairness metrics to evaluate model fairness.


In [ ]:
# Create sample predictions with some bias
np.random.seed(42)
n_samples = 500

y_true = np.random.choice([0, 1], n_samples)
protected_attr = np.random.choice(['group_A', 'group_B'], n_samples, p=[0.6, 0.4])

# Create biased predictions
y_pred = np.zeros(n_samples)
for i in range(n_samples):
    if protected_attr[i] == 'group_A':
        y_pred[i] = np.random.choice([0, 1], p=[0.4, 0.6])
    else:
        y_pred[i] = np.random.choice([0, 1], p=[0.7, 0.3])
    if y_true[i] == 1:
        y_pred[i] = y_true[i]  # Some correlation with true labels

# Initialize fairness metrics
metrics = FairnessMetrics()

# Evaluate fairness
fairness_results = metrics.evaluate(
    y_true=y_true,
    y_pred=y_pred,
    protected_attributes={'gender': protected_attr}
)

print("Fairness Results:")
for attr, results in fairness_results.items():
    print(f"\n  {attr}:")
    if 'demographic_parity' in results:
        dp = results['demographic_parity']
        print(f"    Demographic Parity:")
        print(f"      Parity ratio: {dp['parity_ratio']:.2f}")
        print(f"      Is fair: {dp['is_fair']}")
    
    if 'equal_opportunity' in results:
        eo = results['equal_opportunity']
        print(f"    Equal Opportunity:")
        print(f"      TPR violation: {eo['violation']:.3f}")
        print(f"      Is fair: {eo['is_fair']}")


## 3. Transparency Analysis

Assess model transparency and interpretability.


In [ ]:
from sklearn.datasets import make_classification

# Create and train a model
X, y = make_classification(n_samples=100, n_features=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

# Initialize transparency analyzer
analyzer = TransparencyAnalyzer()

# Assess transparency
transparency_report = analyzer.assess(
    model=model,
    X=X_test[:10],
    feature_names=[f'feature_{i}' for i in range(5)],
    has_documentation=True,
    has_explanations=False
)

print(f"Model Type: {transparency_report['model_type']}")
print(f"Interpretability Score: {transparency_report['interpretability_score']:.2f}")
print(f"Overall Transparency Score: {transparency_report['transparency_score']:.2f}")

print("\nRecommendations:")
for rec in transparency_report['recommendations']:
    print(f"  - {rec}")
